In this tutorial, I will demonstrate the usage of SExtractor background estimation. SExtractor has difficult algorithm and requires more computation time than usual annulus sky estimation, but is convenient in Python, so let me first demonstrate SExtractor sky (background) estimation.



In [4]:
from photutils import Background2D, SigmaClip, MedianBackground, SExtractorBackground
from astropy.io import fits
import numpy as np
hdu = fits.open('HST_Tutorial/M13.fits')
img = hdu[0].data[900:1200, 900:1200]
FWHM = 2.5

box = np.array((50, 50)).astype(int)
filt = (2*FWHM, 2*FWHM)
sigma_clip = SigmaClip(sigma=3., iters=10)

bkg_est_med = MedianBackground()
bkg_est_sex = SExtractorBackground()

bkg_med = Background2D(img, box, filter_size=filt,
                   sigma_clip=sigma_clip, bkg_estimator=bkg_est_med)
bkg_sex = Background2D(img, box, filter_size=filt,
                   sigma_clip=sigma_clip, bkg_estimator=bkg_est_sex)
bkg_diff = (bkg_med.background - bkg_sex.background)/bkg_sex.background * 100

img_sex_diff = (img - bkg_sex.background)/img * 100

print('method | bkg med   rms_med')
print('-------+-----------------')
print('median | {0:7.1f}  {1:7.1f}'.format(bkg_med.background_median, bkg_med.background_rms_median))
print('sex    | {0:7.1f}  {1:7.1f}'.format(bkg_sex.background_median, bkg_sex.background_rms_median))


fig = plt.figure(figsize=(10,10), dpi=80)

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax4.get_yticklabels(), visible=False)
plt.setp((ax1, ax2, ax3, ax4),
         xlim=[0, 400], ylim=[0, 400],
         xticks=np.arange(0, 401, 100),
         yticks=np.arange(0, 401, 100))



#ax1.set_xticks(np.array([0, 100, 200, 300, 400]))
plt.suptitle('estimators used (50, 50) box with (2*FWHM, 2*FWHM) filter, 3-sigma clip (iter=10)', fontsize=16)
im1 = ax1.imshow(bkg_sex.background, origin='lower', cmap='gray_r')
im2 = ax2.imshow(bkg_med.background, origin='lower', cmap='gray_r')
im3 = ax3.imshow(bkg_diff, origin='lower', cmap='gray_r')
im4 = ax4.imshow(img_sex_diff, origin='lower', cmap='gray_r', vmin=10)
ax1.set_title('sextractor estimator', fontsize=16, y=1.03)
ax2.set_title('median estimator', fontsize=16, y=1.03)
ax3.set_title('difference: (med - sex)/sex', fontsize=16, y=1.03)
ax4.set_title('(img - sex)/img', fontsize=16, y=1.03)
ax1.tick_params(labelsize=16)
ax2.tick_params(labelsize=16)
ax3.tick_params(labelsize=16)
ax4.tick_params(labelsize=16)

cbar_ax12 = fig.add_axes([0.95, 0.55, 0.01, 0.35]) # left, bottom, width, height
cbar_ax3  = fig.add_axes([0.12, 0.08, 0.35, 0.01]) # left, bottom, width, height
cbar_ax4  = fig.add_axes([0.95, 0.12, 0.01, 0.35]) # left, bottom, width, height
fig.colorbar(im1, cax=cbar_ax12)
fig.colorbar(im3, cax=cbar_ax3,
             orientation='horizontal',
             label='%')
fig.colorbar(im4, cax=cbar_ax4,
             extend='min',
             label='% (if <10%, greenish)').cmap.set_under('green', alpha=0.5)
# If (img-bkg)/img < 10%, the pixel is likely a meaningless background.
plt.savefig('bkg_sex+med.png', bbox_inches='tight')
plt.clf()

fig = plt.figure(figsize=(10,10), dpi=80)

plt.imshow(bkg_sex.background, origin='lower')
bkg_sex.plot_meshes(outlines=True)
plt.colorbar()


TypeError: 'numpy.float64' object cannot be interpreted as an integer